# Extractive Summarization

## Load necessary modules

In [1]:
from datasets import load_dataset
import pandas as pd
import optuna

from Dataset import Dataset

## Load CNN\_dailymail dataset and initiate Dataset data structure

In [2]:
# Load dataset into a variable
CNN_dataset = load_dataset('cnn_dailymail', '3.0.0')

# Create a new instance of the Dataset class with a custom name
# CNN_processed = Dataset(name='CNN_processed.json')

Reusing dataset cnn_dailymail (/home/davide/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


  0%|          | 0/3 [00:00<?, ?it/s]

## Meena and Gopalani strategy comparison

In [11]:
# Meena & Gopalani environment
MG_scores = {
             'comb1': ['TF_ISF_IDF', 'co_occur', 'sent_length'],
             'comb2': ['co_occur', 'sent_length', 'sent_location'],
             'comb3': ['TF_ISF_IDF', 'co_occur', 'sent_length',
                       'sent_location'],
             'comb4': ['sent_length', 'sent_location', 'named_entities',
                       'pos_keywords', 'proper_noun'],
             'comb5': ['co_occur', 'sent_length', 'sent_location',
                       'named_entities', 'pos_keywords', 'proper_noun'],
             'comb6': ['TF_ISF_IDF', 'co_occur', 'sent_length',
                       'sent_location', 'named_entities', 'pos_keywords',
                       'neg_keywords', 'sent_rank'],
             'comb7': ['TF_ISF_IDF', 'co_occur', 'sent_length',
                       'sent_location', 'named_entities', 'pos_keywords',
                       'neg_keywords']
             }

# Meena & Golapani initial test
MG_test = Dataset(name='MG_test_dataset.json')
MG_num_docs = 100
MG_test.build_dataset(CNN_dataset['train'], MG_num_docs)
MG_results = pd.DataFrame()
for comb, scores in MG_scores.items():
    MG_test.process_dataset(scoreList=scores)
    MG_rouge = MG_test.rouge_computation(show=False)
    MG_results = pd.concat([MG_results, MG_rouge.loc['Mean']], axis=1,
                           ignore_index=True)
print(MG_results.T)

processing dataset: :   0%|                                         | 27/287113 [00:05<13:44:51,  5.80it/s]/home/davide/Desktop/NLU_Final_Project/Dataset.py:105: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  sentence_similarity = sentence.similarity(sent2)
processing dataset: :   0%|                                        | 100/287113 [00:17<14:15:38,  5.59it/s]


Dataset built in 18.49911141395569[sec]


computing scores: : 100%|████████████████████████████████████████████████| 100/100 [00:01<00:00, 59.28it/s]


Dataset processed in: 1.6881[sec]


computing scores: : 100%|███████████████████████████████████████████████| 100/100 [00:00<00:00, 104.89it/s]


...

Dataset processed in: 1.8458[sec]


computing scores: : 100%|████████████████████████████████████████████████| 100/100 [00:03<00:00, 30.57it/s]


Dataset processed in: 3.2728[sec]


computing scores: : 100%|████████████████████████████████████████████████| 100/100 [00:03<00:00, 32.32it/s]

Dataset processed in: 3.0951[sec]
    Rouge-2  Precision  F1-score
0  0.114578   0.035599  0.053768
1  0.070260   0.048146  0.055320
2  0.117600   0.036917  0.055564
3  0.146901   0.075990  0.097490
4  0.144287   0.074426  0.095598
5  0.120043   0.037519  0.056511
6  0.119123   0.037252  0.056104


## All scores computation

A warning is flagged in processing document 27 due to some of its token being unknown by spacy

In [9]:
results = pd.DataFrame(columns=['Rouge-2', 'Precision', 'F1-score'])
for _doc_num in [6, 100, 1000]:
    CNN_processed.process_dataset(CNN_dataset['train'], doc_th=_doc_num)
    res = CNN_processed.rouge_computation(show=False)
    results.loc[_doc_num] = res.loc['Mean'].T
print(results)

computing scores: : 100%|████████████████████████████████████████████████████| 6/6 [00:00<00:00, 23.73it/s]


Dataset processed in: 0.2541[sec]


processing dataset: :   0%|                                         | 27/287113 [00:04<10:20:07,  7.72it/s]/home/davide/Desktop/NLU_Final_Project/Dataset.py:105: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  sentence_similarity = sentence.similarity(sent2)
computing scores: : 100%|████████████████████████████████████████████████| 100/100 [00:04<00:00, 24.16it/s]


Dataset processed in: 4.1399[sec]


computing scores: : 100%|██████████████████████████████████████████████| 1000/1000 [02:11<00:00,  7.63it/s]


Dataset processed in: 131.1135[sec]
       Rouge-2  Precision  F1-score
6     0.109382   0.028958  0.045368
100   0.120221   0.037403  0.056382
1000  0.123249   0.038178  0.057659


## Experiment: Maximum among all the computed Sentence location scores 

In [15]:
# Sum of Location scoring strategies
CNN_processed = Dataset('CNN_dataset')
CNN_processed.process_dataset(CNN_dataset['train'], doc_th=100,
                              all_loc_scores=True)
result = CNN_processed.rouge_computation()

processing dataset: :   0%|                                         | 27/287113 [00:04<10:23:01,  7.68it/s]/home/davide/Desktop/NLU_Final_Project/Dataset.py:105: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  sentence_similarity = sentence.similarity(sent2)
computing scores: : 100%|████████████████████████████████████████████████| 100/100 [00:04<00:00, 23.96it/s]

Dataset processed in: 4.1745[sec]
                                           Rouge-2  Precision  F1-score
0001d1afc246a7964130f43ae940af6bc6c57f01  0.120000   0.027650  0.044944
0002095e55fcbd3a2f366d9bf92a95433dc305ef  0.111111   0.017964  0.030928
00027e965c8264c35cc1bc55556db388da82b07f  0.027027   0.007246  0.011429
0002c17436637c4fe1837c935c04de47adb18e9a  0.062500   0.016760  0.026432
0003ad6ef0c37534f80b55b4235108024b407f0b  0.031250   0.010526  0.015748
...                                            ...        ...       ...
004f0f8c694c4b546b29565a8993a555537ff561  0.054054   0.016260  0.025000
004fc12e7cd2505a013d96e816afae3f3ce5015d  0.258065   0.109589  0.153846
00504275ede73591d94a6c1f994fd4856610421c  0.019608   0.007812  0.011173
00512126d65bf2a36801e4ef37f28c86c29deb28  0.085106   0.021053  0.033755
Mean                                      0.122133   0.038602  0.057941

[101 rows x 3 columns]


## Experiment: Location scoring with threshold analysis

In [ ]:
# Nobata Location Treshold analysis
CNN_processed = Dataset('CNN_dataset')
CNN_processed.build_dataset(CNN_dataset['train'], doc_th=100)
loc_task = pd.DataFrame(columns=['Rouge-2', 'Precision', 'F1-score'])
for x in range(1, 21):
    CNN_processed.process_dataset(loc_th=x, all_loc_scores=True)
    loc_task.loc[x] = CNN_processed.rouge_computation(show=False).loc['Mean']
print(loc_task)

## Experiment: Location Scoring with treshold analysis using only Nobata et al. method

In [ ]:
# Nobata Location Treshold analysis part 2
CNN_processed = Dataset('CNN_dataset')
CNN_processed.build_dataset(CNN_dataset['train'], doc_th=100)
loc_task = pd.DataFrame(columns=['Rouge-2', 'Precision', 'F1-score'])
for x in range(1, 21):
    CNN_processed.process_dataset(loc_th=x, locFilter=[0, 1, 0, 0, 0])  # Only NB1 used
    loc_task.loc[x] = CNN_processed.rouge_computation(show=False).loc['Mean']
print(loc_task)

## Experiment: Lemma Usage

In [25]:
results = pd.DataFrame(columns=['Rouge-2', 'Precision', 'F1-score'])
CNN_processed = Dataset('CNN_dataset')
# CNN_processed.build_dataset(CNN_dataset['train'], doc_th=100)

_doc_num = 100
for _lemma in [False, True]:
    CNN_processed.process_dataset(CNN_dataset['train'], doc_th=100,
                                  lemma=_lemma, all_loc_scores=True)
    res = CNN_processed.rouge_computation(show=False)
    results.loc[str(_lemma)] = res.loc['Mean'].T
print(results)

processing dataset: :   0%|                                         | 27/287113 [00:04<10:21:07,  7.70it/s]/home/davide/Desktop/NLU_Final_Project/Dataset.py:105: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  sentence_similarity = sentence.similarity(sent2)
computing scores: : 100%|████████████████████████████████████████████████| 100/100 [00:04<00:00, 24.14it/s]


Dataset processed in: 4.1430[sec]


computing scores: : 100%|████████████████████████████████████████████████| 100/100 [00:04<00:00, 24.51it/s]

Dataset processed in: 4.0817[sec]
        Rouge-2  Precision  F1-score
False  0.122133   0.038602  0.057941
True   0.156094   0.059743  0.085454


## Define optimization task functions

In [3]:
def CNN_scores_best_representation(trial, CNN_dataset, value, pipeline=None, lemma=False):
    scores = set()
    available_scores = CNN_dataset.get_num_weights(True)  # Only names
    
    i = trial.suggest_int('Number_of_scores', 1, len(available_scores))
    for x in range(i):
        score = trial.suggest_categorical('Score-%d' % x, available_scores)
        scores.add(score)

    # CNN_dataset.process_dataset(scoreList=scores, all_loc_scores=True, lemma=lemma, nlp=pipeline)
    for doc in CNN_dataset.documents.values():
        for sentence in doc.sentences.values():
            sent_score = sentence.scores.__dict__.keys()
            for name in sent_score:
                if name not in scores:
                    sentence.scores.__dict__[name] = 0
    
    results = CNN_dataset.rouge_computation(show=False)  # make rouge computation call the scores
    return results.loc['Mean'][value]


def CNN_scores_weights_task(trial, CNN_dataset, value, pipeline=None, lemma=False):
    scores = [x for x in CNN_dataset.get_num_weights(True)]
    weights = [1 for x in range(len(scores))]

    for score in scores:
        idx = scores.index(score)
        weights[idx] = trial.suggest_float(score, -10.0, 10.0, step=0.5)

    results = CNN_dataset.rouge_computation(weights=weights, show=False)
    return results.loc['Mean'][value]

## Experiment: Best Weights with and Without lemma usage optimization task

In [16]:
# Scores weights optimization task -> results may change due to stochasticity in the optimizer
best_weights_res = pd.DataFrame(data=[], columns=['Rouge-2', 'Precision', 'F1-score'])
weights9 = {'False': {}, 'True': {}}

for _lemma in [False, True]:
    CNN_dataset = load_dataset('cnn_dailymail', '3.0.0')
    CNN_processed = Dataset('Weights_finding_optimisation_task.json')
    
    CNN_processed.process_dataset(CNN_dataset['train'], doc_th=100, all_loc_scores=True, lemma=_lemma)
    
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: CNN_scores_weights_task(trial, CNN_processed, 'Precision', lemma=_lemma),
                   n_trials=500)

    weights = list(study.best_params.values())

    weight_res = CNN_processed.rouge_computation(show=False, weights=weights)
    
    best_weights_res.loc[str(_lemma)] = weight_res.loc['Mean'].T
    
    best_param = study.best_params.copy()
    
    for key, value in best_param.items():
        if value >= 9:
            weights9[str(_lemma)].update({key: value})
    
    CNN_processed.process_dataset(lemma=_lemma, scoreList=list(weights9[str(_lemma)].keys()))
    weight_9_rouge = CNN_processed.rouge_computation(show=False, weights=weights)
    
    best_weights_res.loc[str(_lemma)+'_9'] = weight_9_rouge.loc['Mean'].T
print(best_weights_res)
print(weights9)

Reusing dataset cnn_dailymail (/home/davide/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


  0%|          | 0/3 [00:00<?, ?it/s]

processing dataset: :   0%|                                       | 27/287113 [00:05<14:25:04,  5.53it/s]/home/davide/Desktop/NLU_Final_Project/Dataset.py:213: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  sentence_similarity = sentence.similarity(sent2)
computing scores: : 100%|██████████████████████████████████████████████| 100/100 [00:06<00:00, 16.41it/s]
[I 2021-12-28 00:42:35,576] A new study created in memory with name: no-name-e505b80f-02db-4237-8842-70de93694ff8
[I 2021-12-28 00:42:35,614] Trial 0 finished with value: 0.024003465661074332 and parameters: {'TF': -9.5, 'sent_location': -8.0, 'proper_noun': 7.0, 'co_occur': -7.5, 'sent_similarity': 5.5, 'num_val': -3.5, 'TF_ISF_IDF': -7.0, 'sent_rank': 7.5, 'sent_length': -6.0, 'pos_keywords': -3.0, 'neg_keywords': 8.5, 'thematic_features': -5.5, 'named_entities': -0.5}. Best is trial 0 with value: 0.024003465661074332.
[I 2021-12-28 00:42:35,646] Trial 1 finished with value: 0.014501938162687238 and par

Dataset processed in: 6.0943[sec]


[I 2021-12-28 00:42:35,801] Trial 4 finished with value: 0.03242217702072935 and parameters: {'TF': -1.5, 'sent_location': -6.5, 'proper_noun': 7.0, 'co_occur': 9.5, 'sent_similarity': 0.0, 'num_val': 6.5, 'TF_ISF_IDF': 4.5, 'sent_rank': -5.5, 'sent_length': -6.5, 'pos_keywords': 7.5, 'neg_keywords': -4.0, 'thematic_features': 6.5, 'named_entities': -7.5}. Best is trial 3 with value: 0.03501224946328171.
[I 2021-12-28 00:42:35,837] Trial 5 finished with value: 0.03671079663142853 and parameters: {'TF': -8.0, 'sent_location': 8.5, 'proper_noun': -8.5, 'co_occur': 4.0, 'sent_similarity': -7.5, 'num_val': 8.0, 'TF_ISF_IDF': -6.0, 'sent_rank': -3.5, 'sent_length': 2.0, 'pos_keywords': 6.5, 'neg_keywords': 4.0, 'thematic_features': -7.0, 'named_entities': 6.5}. Best is trial 5 with value: 0.03671079663142853.
[I 2021-12-28 00:42:35,875] Trial 6 finished with value: 0.022656813623072834 and parameters: {'TF': 7.0, 'sent_location': 1.0, 'proper_noun': -1.5, 'co_occur': 2.5, 'sent_similarity':

...

[I 2021-12-28 00:44:12,454] Trial 424 finished with value: 0.13083761745651376 and parameters: {'TF': 4.5, 'sent_location': 8.0, 'proper_noun': 5.5, 'co_occur': 8.0, 'sent_similarity': 3.5, 'num_val': -0.5, 'TF_ISF_IDF': 0.0, 'sent_rank': -2.5, 'sent_length': 4.5, 'pos_keywords': 9.0, 'neg_keywords': 10.0, 'thematic_features': -4.5, 'named_entities': -7.0}. Best is trial 410 with value: 0.20181025416094037.
[I 2021-12-28 00:44:12,539] Trial 425 finished with value: 0.05950461308435681 and parameters: {'TF': 5.0, 'sent_location': 2.5, 'proper_noun': -8.5, 'co_occur': 6.5, 'sent_similarity': 2.0, 'num_val': 1.0, 'TF_ISF_IDF': 9.0, 'sent_rank': -3.0, 'sent_length': 4.0, 'pos_keywords': 9.0, 'neg_keywords': 9.5, 'thematic_features': -5.0, 'named_entities': -8.0}. Best is trial 410 with value: 0.20181025416094037.
[I 2021-12-28 00:44:12,616] Trial 426 finished with value: 0.19171655147087577 and parameters: {'TF': 6.0, 'sent_location': 3.0, 'proper_noun': 2.5, 'co_occur': 7.5, 'sent_similar

[I 2021-12-28 00:44:14,020] Trial 444 finished with value: 0.06219588140228798 and parameters: {'TF': 4.0, 'sent_location': 2.0, 'proper_noun': 4.5, 'co_occur': 9.5, 'sent_similarity': 4.0, 'num_val': 2.0, 'TF_ISF_IDF': -0.5, 'sent_rank': 0.0, 'sent_length': 3.5, 'pos_keywords': -4.0, 'neg_keywords': 1.5, 'thematic_features': -5.0, 'named_entities': -6.0}. Best is trial 410 with value: 0.20181025416094037.
[I 2021-12-28 00:44:14,096] Trial 445 finished with value: 0.19566906482030863 and parameters: {'TF': 3.0, 'sent_location': 1.5, 'proper_noun': 4.0, 'co_occur': 10.0, 'sent_similarity': 3.5, 'num_val': 3.0, 'TF_ISF_IDF': 0.0, 'sent_rank': 2.0, 'sent_length': 3.0, 'pos_keywords': 9.0, 'neg_keywords': 9.0, 'thematic_features': -6.0, 'named_entities': -6.5}. Best is trial 410 with value: 0.20181025416094037.
[I 2021-12-28 00:44:14,176] Trial 446 finished with value: 0.15337676303999717 and parameters: {'TF': 3.5, 'sent_location': 1.5, 'proper_noun': 3.0, 'co_occur': 7.0, 'sent_similarit

[I 2021-12-28 00:44:15,606] Trial 464 finished with value: 0.17742567479746982 and parameters: {'TF': 6.5, 'sent_location': 1.0, 'proper_noun': 5.5, 'co_occur': 8.5, 'sent_similarity': 3.5, 'num_val': 1.5, 'TF_ISF_IDF': -0.5, 'sent_rank': 1.5, 'sent_length': 3.5, 'pos_keywords': 9.5, 'neg_keywords': 9.5, 'thematic_features': -5.5, 'named_entities': -4.5}. Best is trial 410 with value: 0.20181025416094037.
[I 2021-12-28 00:44:15,682] Trial 465 finished with value: 0.13429667698546177 and parameters: {'TF': 3.0, 'sent_location': 0.5, 'proper_noun': -5.5, 'co_occur': 6.5, 'sent_similarity': 8.0, 'num_val': 2.0, 'TF_ISF_IDF': -0.5, 'sent_rank': 2.0, 'sent_length': 4.0, 'pos_keywords': 9.0, 'neg_keywords': 9.0, 'thematic_features': -5.0, 'named_entities': -8.0}. Best is trial 410 with value: 0.20181025416094037.
[I 2021-12-28 00:44:15,765] Trial 466 finished with value: 0.1511592859002903 and parameters: {'TF': 5.5, 'sent_location': -0.5, 'proper_noun': 5.0, 'co_occur': 9.5, 'sent_similarit

[I 2021-12-28 00:44:17,228] Trial 484 finished with value: 0.08241434468253508 and parameters: {'TF': 3.0, 'sent_location': 1.5, 'proper_noun': 6.5, 'co_occur': 7.5, 'sent_similarity': 2.5, 'num_val': -0.5, 'TF_ISF_IDF': 1.5, 'sent_rank': 2.0, 'sent_length': 3.0, 'pos_keywords': 10.0, 'neg_keywords': 10.0, 'thematic_features': -3.0, 'named_entities': -6.0}. Best is trial 410 with value: 0.20181025416094037.
[I 2021-12-28 00:44:17,303] Trial 485 finished with value: 0.10151264741714047 and parameters: {'TF': 3.5, 'sent_location': 2.5, 'proper_noun': 5.0, 'co_occur': 6.5, 'sent_similarity': 3.0, 'num_val': 4.0, 'TF_ISF_IDF': -4.0, 'sent_rank': 0.5, 'sent_length': 4.0, 'pos_keywords': 9.5, 'neg_keywords': 9.5, 'thematic_features': -4.0, 'named_entities': -7.0}. Best is trial 410 with value: 0.20181025416094037.
[I 2021-12-28 00:44:17,379] Trial 486 finished with value: 0.16921199405984974 and parameters: {'TF': 4.0, 'sent_location': 2.0, 'proper_noun': -6.5, 'co_occur': 8.0, 'sent_similar

Dataset processed in: 3.0055[sec]
          Rouge-2  Precision  F1-score
False    0.185936   0.147467  0.153839
False_9  0.228199   0.138573  0.165973
True     0.252152   0.201810  0.213269
True_9   0.275119   0.216091  0.232243
{'False': {'pos_keywords': 9.0, 'neg_keywords': 10.0}, 'True': {'pos_keywords': 9.0, 'neg_keywords': 10.0}}


## Experiment: Best Representing subset, optimization task

Takes 5 minutes c.a

In [5]:
CNN_dataset = load_dataset('cnn_dailymail', '3.0.0')
CNN_processed = Dataset('Weights_finding_optimisation_task.json')

subset_rouge = pd.DataFrame(columns=['Rouge-2', 'Precision', 'F1-score'])
subsets = {}
_num_doc = 100

for _lemma in [False, True]:
    # pipe = CNN_processed.build_dataset(CNN_dataset['train'], doc_th=_num_doc,
    #                                    return_pipe=True, lemma=_lemma, suppress_warnings=True)
    CNN_processed.process_dataset(CNN_dataset['train'], doc_th=_num_doc,
                                  lemma=_lemma, suppress_warnings=True)
    for _value in ['Precision', 'Rouge-2', 'F1-score']:
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: CNN_scores_best_representation(trial,
                                                                    CNN_processed,
                                                                    _value,
                                                                    # pipe,
                                                                    lemma=_lemma),
                       n_trials=500)

        subset = set()
        for key, value in study.best_params.items():
            if key != 'Number_of_scores':
                subset.add(value)
        
        CNN_processed.process_dataset(lemma=_lemma, scoreList=subset)
        index = _value + '_Lemma_{}'.format(_lemma)
        subsets[index] = subset
        subset_rouge.loc[index] = CNN_processed.rouge_computation(show=False).loc['Mean'].T
    CNN_processed.__init__('Weights_finding_optimisation_task.json')
print(subset_rouge, '\n')

for key, value in subsets.items():
    print(key, '\t', value)

Reusing dataset cnn_dailymail (/home/davide/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


  0%|          | 0/3 [00:00<?, ?it/s]

processing dataset: :   0%|                                       | 27/287113 [00:05<17:27:23,  4.57it/s]/home/davide/Desktop/NLU_Final_Project/Dataset.py:213: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  sentence_similarity = sentence.similarity(sent2)
computing scores: : 100%|██████████████████████████████████████████████| 100/100 [00:05<00:00, 19.11it/s]
[I 2021-12-28 01:33:51,969] A new study created in memory with name: no-name-7d5b5e0b-54be-4888-9bf0-2d7ca416a6be
[I 2021-12-28 01:33:51,999] Trial 0 finished with value: 0.10409090756458933 and parameters: {'Number_of_scores': 10, 'Score-0': 'TF', 'Score-1': 'pos_keywords', 'Score-2': 'num_val', 'Score-3': 'neg_keywords', 'Score-4': 'thematic_features', 'Score-5': 'num_val', 'Score-6': 'pos_keywords', 'Score-7': 'sent_similarity', 'Score-8': 'named_entities', 'Score-9': 'num_val'}. Best is trial 0 with value: 0.10409090756458933.
[I 2021-12-28 01:33:52,027] Trial 1 finished with value: 0.0829033315708816

Dataset processed in: 5.2338[sec]


[I 2021-12-28 01:33:52,219] Trial 8 finished with value: 0.05701359796322159 and parameters: {'Number_of_scores': 1, 'Score-0': 'TF'}. Best is trial 0 with value: 0.10409090756458933.
[I 2021-12-28 01:33:52,247] Trial 9 finished with value: 0.05701359796322159 and parameters: {'Number_of_scores': 12, 'Score-0': 'sent_similarity', 'Score-1': 'num_val', 'Score-2': 'neg_keywords', 'Score-3': 'named_entities', 'Score-4': 'sent_location', 'Score-5': 'sent_location', 'Score-6': 'sent_rank', 'Score-7': 'TF', 'Score-8': 'sent_length', 'Score-9': 'sent_similarity', 'Score-10': 'named_entities', 'Score-11': 'sent_similarity'}. Best is trial 0 with value: 0.10409090756458933.
[I 2021-12-28 01:33:52,280] Trial 10 finished with value: 0.05701359796322159 and parameters: {'Number_of_scores': 8, 'Score-0': 'num_val', 'Score-1': 'pos_keywords', 'Score-2': 'num_val', 'Score-3': 'neg_keywords', 'Score-4': 'thematic_features', 'Score-5': 'num_val', 'Score-6': 'pos_keywords', 'Score-7': 'sent_similarity'}

...

[I 2021-12-28 01:36:17,268] Trial 404 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 10, 'Score-0': 'thematic_features', 'Score-1': 'co_occur', 'Score-2': 'sent_similarity', 'Score-3': 'TF', 'Score-4': 'thematic_features', 'Score-5': 'sent_rank', 'Score-6': 'sent_rank', 'Score-7': 'sent_similarity', 'Score-8': 'num_val', 'Score-9': 'TF_ISF_IDF'}. Best is trial 1 with value: 0.2209531210570851.
[I 2021-12-28 01:36:17,311] Trial 405 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 11, 'Score-0': 'num_val', 'Score-1': 'neg_keywords', 'Score-2': 'thematic_features', 'Score-3': 'TF', 'Score-4': 'thematic_features', 'Score-5': 'thematic_features', 'Score-6': 'named_entities', 'Score-7': 'co_occur', 'Score-8': 'num_val', 'Score-9': 'TF_ISF_IDF', 'Score-10': 'sent_rank'}. Best is trial 1 with value: 0.2209531210570851.
[I 2021-12-28 01:36:17,353] Trial 406 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 1

[I 2021-12-28 01:36:18,025] Trial 422 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 11, 'Score-0': 'sent_rank', 'Score-1': 'sent_similarity', 'Score-2': 'named_entities', 'Score-3': 'sent_location', 'Score-4': 'TF', 'Score-5': 'num_val', 'Score-6': 'named_entities', 'Score-7': 'co_occur', 'Score-8': 'TF_ISF_IDF', 'Score-9': 'sent_rank', 'Score-10': 'named_entities'}. Best is trial 1 with value: 0.2209531210570851.
[I 2021-12-28 01:36:18,068] Trial 423 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 11, 'Score-0': 'sent_location', 'Score-1': 'sent_rank', 'Score-2': 'sent_similarity', 'Score-3': 'sent_length', 'Score-4': 'thematic_features', 'Score-5': 'num_val', 'Score-6': 'num_val', 'Score-7': 'co_occur', 'Score-8': 'TF', 'Score-9': 'pos_keywords', 'Score-10': 'pos_keywords'}. Best is trial 1 with value: 0.2209531210570851.
[I 2021-12-28 01:36:18,111] Trial 424 finished with value: 0.10086838478308793 and parameters: {'Numbe

[I 2021-12-28 01:36:18,800] Trial 440 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 10, 'Score-0': 'sent_length', 'Score-1': 'TF', 'Score-2': 'co_occur', 'Score-3': 'named_entities', 'Score-4': 'sent_similarity', 'Score-5': 'thematic_features', 'Score-6': 'thematic_features', 'Score-7': 'pos_keywords', 'Score-8': 'thematic_features', 'Score-9': 'thematic_features'}. Best is trial 1 with value: 0.2209531210570851.
[I 2021-12-28 01:36:18,843] Trial 441 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 10, 'Score-0': 'sent_length', 'Score-1': 'TF', 'Score-2': 'sent_similarity', 'Score-3': 'named_entities', 'Score-4': 'sent_similarity', 'Score-5': 'num_val', 'Score-6': 'thematic_features', 'Score-7': 'proper_noun', 'Score-8': 'named_entities', 'Score-9': 'pos_keywords'}. Best is trial 1 with value: 0.2209531210570851.
[I 2021-12-28 01:36:18,885] Trial 442 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores'

[I 2021-12-28 01:36:19,821] Trial 468 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 5, 'Score-0': 'TF', 'Score-1': 'sent_rank', 'Score-2': 'sent_location', 'Score-3': 'TF_ISF_IDF', 'Score-4': 'sent_length'}. Best is trial 1 with value: 0.2209531210570851.
[I 2021-12-28 01:36:19,859] Trial 469 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 5, 'Score-0': 'sent_similarity', 'Score-1': 'thematic_features', 'Score-2': 'sent_location', 'Score-3': 'TF_ISF_IDF', 'Score-4': 'sent_length'}. Best is trial 1 with value: 0.2209531210570851.
[I 2021-12-28 01:36:19,896] Trial 470 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 5, 'Score-0': 'sent_similarity', 'Score-1': 'sent_rank', 'Score-2': 'sent_location', 'Score-3': 'TF_ISF_IDF', 'Score-4': 'sent_length'}. Best is trial 1 with value: 0.2209531210570851.
[I 2021-12-28 01:36:19,931] Trial 471 finished with value: 0.10086838478308793 and parameters: {'Numbe

Dataset processed in: 2.1983[sec]
                        Rouge-2  Precision  F1-score
Precision_Lemma_False  0.157597   0.104091  0.115773
Rouge-2_Lemma_False    0.075745   0.046878  0.054350
F1-score_Lemma_False   0.115608   0.035662  0.053936
Precision_Lemma_True   0.140301   0.056237  0.079395
Rouge-2_Lemma_True     0.186459   0.081522  0.111995
F1-score_Lemma_True    0.168778   0.122872  0.139372 

Precision_Lemma_False 	 {'TF', 'named_entities', 'num_val', 'pos_keywords', 'thematic_features', 'sent_similarity', 'neg_keywords'}
Rouge-2_Lemma_False 	 {'sent_length', 'sent_similarity', 'sent_rank', 'named_entities'}
F1-score_Lemma_False 	 {'TF', 'named_entities', 'pos_keywords', 'TF_ISF_IDF', 'sent_similarity'}
Precision_Lemma_True 	 {'sent_length', 'TF_ISF_IDF', 'named_entities'}
Rouge-2_Lemma_True 	 {'sent_rank', 'neg_keywords', 'proper_noun'}
F1-score_Lemma_True 	 {'sent_length', 'TF', 'num_val', 'thematic_features', 'neg_keywords'}
